In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
base_path = Path().resolve().parent
%cd {base_path}

/home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
# setup
#!apt-get update
#!pip install python-dotenv
#!pip install loguru
#!pip install gdown
#!pip install typer
#!pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.4 MB/s eta 0:00:00:00:01


In [4]:
# load data and unzip data

#!python tools/download_data_and_chkpts.py

In [29]:
# imports

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import wandb

from sklearn.metrics import roc_auc_score, average_precision_score

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import time

# data
from ails_miccai_uwf4dr_challenge.dataset_strategy import CustomDataset, DatasetStrategy, CombinedDatasetStrategy, \
    Task1Strategy, Task2Strategy, Task3Strategy, TrainValSplitStrategy, RandomOverSamplingStrategy, DatasetBuilder

# augmentation
from ails_miccai_uwf4dr_challenge.preprocess_augmentations import ResidualGaussBlur, MultiplyMask

# trainer
from ails_miccai_uwf4dr_challenge.models.trainer import DefaultMetricsEvaluationStrategy, Metric, MetricCalculatedHook, \
    NumBatches, Trainer, EpochTrainingStrategy, EpochValidationStrategy, DefaultEpochTrainingStrategy, \
    DefaultBatchTrainingStrategy, TrainingContext, PersistBestModelOnEpochEndHook
from ails_miccai_uwf4dr_challenge.models.metrics import sensitivity_score, specificity_score
from ails_miccai_uwf4dr_challenge.config import Config

# models
from ails_miccai_uwf4dr_challenge.models.architectures.task1_automorph_plain import AutoMorphModel
from ails_miccai_uwf4dr_challenge.models.architectures.task1_efficientnet_plain import Task1EfficientNetB4
from ails_miccai_uwf4dr_challenge.models.architectures.task2_efficientnetb0_plain import Task2EfficientNetB0 
from ails_miccai_uwf4dr_challenge.models.architectures.task1_convnext import Task1ConvNeXt 

from ails_miccai_uwf4dr_challenge.config import WANDB_API_KEY, PROJ_ROOT

wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/optima/mhaderer/.netrc


True

In [24]:
# select device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

Device: cuda


## Train some model

In [28]:
def train_model(cfg=None):

    WANDB_HTTP_TIMEOUT=300
    WANDB_INIT_TIMEOUT =600
    WANDB_DEBUG=True

    wandb.init(project="task2", config=cfg)
    config = wandb.config

    transforms_train = A.Compose([
        A.Resize(800, 1016, p=1),
        MultiplyMask(p=1),
        ResidualGaussBlur(p=cfg.p_gaussblur),
        A.Equalize(p=cfg.p_equalize),
        A.CLAHE(clip_limit=5., p=cfg.p_clahe),
        A.HorizontalFlip(p=cfg.p_horizontalflip),
        A.Affine(rotate=cfg.rotation, rotate_method='ellipse', p=cfg.p_affine),
        A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
        ToTensorV2(p=1)
    ])

    transforms_val = A.Compose([
            A.Resize(800, 1016, p=1),
            MultiplyMask(p=1),
            A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
            ToTensorV2(p=1)
        ])

    dataset_strategy = CombinedDatasetStrategy()
    task_strategy = Task2Strategy()
    
    split_strategy = TrainValSplitStrategy(split_ratio=0.8)
    resampling_strategy = RandomOverSamplingStrategy()

    builder = DatasetBuilder(dataset_strategy, task_strategy, split_strategy, resampling_strategy)
    train_data, val_data = builder.build()

    train_dataset = CustomDataset(train_data, transform=transforms_train)
    val_dataset = CustomDataset(val_data, transform=transforms_val)

    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=cfg.batch_size, shuffle=False)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu" if torch.backends.mps.is_available() else "cpu") #don't use mps, it takes ages, whyever that is the case!?!
    print(f"Using device: {device}")

    assert cfg.model is not None, "Model type must be specified in the config"

    if cfg.model == "Task2EfficientNetB0":
        model = Task2EfficientNetB0(num_classes=1)
    elif cfg.model == "AutoMorphModel":
        model = AutoMorphModel(enc_frozen=True)
    elif cfg.model == "Task1ConvNeXt":
        model = Task1ConvNeXt(num_classes=1)

    model = Task2EfficientNetB0(num_classes=1)

    model.to(device)

    metrics = [
        Metric('auroc', roc_auc_score),
        Metric('auprc', average_precision_score),
        Metric('accuracy', lambda y_true, y_pred: (y_pred.round() == y_true).mean()),
        Metric('sensitivity', sensitivity_score),
        Metric('specificity', specificity_score)
    ]

    class WandbLoggingHook(MetricCalculatedHook):
        def on_metric_calculated(self, training_context: TrainingContext, metric: Metric, result, last_metric_for_epoch: bool):
            import wandb
            wandb.log(data={metric.name: result}, commit=last_metric_for_epoch)

    metrics_eval_strategy = DefaultMetricsEvaluationStrategy(metrics).register_metric_calculated_hook(WandbLoggingHook())

    def combined_losses(pred, target):
        bce = F.binary_cross_entropy_with_logits(pred, target) * cfg.loss_weight
        smooth_l1 = F.smooth_l1_loss(pred, target) * (1 - cfg.loss_weight)
        return bce + smooth_l1

    criterion = combined_losses
    optimizer = optim.AdamW(model.parameters(), lr=cfg.lr)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=cfg.lr_schedule_factor, patience=cfg.lr_schedule_patience, verbose=True)

    trainer = Trainer(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, device, 
                        metrics_eval_strategy=metrics_eval_strategy)

    # build a file name for the model weights containing current timestamp and the model class
    training_timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
    persist_model_hook = PersistBestModelOnEpochEndHook(f"models/best_model_{training_timestamp}.pth")
    trainer.add_epoch_end_hook(persist_model_hook) # TODO uncomment this line to save the best model

    #print("First train 2 epochs 2 batches to check if everything works - you can comment these two lines after the code has stabilized...")
    #trainer.train(num_epochs=2, num_batches=NumBatches.TWO_FOR_INITIAL_TESTING)
    
    print("Now train train train")
    trainer.train(num_epochs=cfg.epochs)
    wandb.finish()
    print("Finished training")

[autoreload of ails_miccai_uwf4dr_challenge.preprocess_augmentations failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge/ails_miccai_uwf4dr_challenge/preprocess_augmentations.py", line 153, in <module>
    if __name__ == __main__:
NameError: name '__main__' is not defined. Did you mean: '__name__'?
]


In [26]:
cfg = Config(
    batch_size=16,
    epochs=20,
    lr=0.001,
    lr_schedule_factor=0.1,
    lr_schedule_patience=5,
    p_gaussblur=0.3,
    p_equalize=0.1,
    p_clahe=0.3,
    p_horizontalflip=0.3,
    rotation=15,
    p_affine=0.3,
    loss_weight=0.5,
    smooth_l1_weight=0.5,
    model=Task2EfficientNetB0,
)

In [27]:
train_model(cfg)


Original class distribution:
Class 0: 149
Class 1: 250
dr
1    0.626566
0    0.373434
Name: proportion, dtype: Float64

Resampled class distribution:
dr
1    0.5
0    0.5
Name: proportion, dtype: Float64
Using device: cuda
Loaded pretrained weights for efficientnet-b0


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Now train train train


Epoch 1/20 - Avg val Loss: 0.948799: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.40s/it]


New best model found at epoch 1 with validation loss: 0.9488. Model saved to models/best_model_2024-07-23_14-49-28.pth


...

New best model found at epoch 12 with validation loss: 0.2860. Model saved to models/best_model_2024-07-23_14-49-28.pth


Epoch 20/20 - Avg val Loss: 0.286323: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.36s/it]


wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.


accuracy,▁▆▆▇▇▆▃▆▇▇▇██▆▇█████
auprc,▁▆▆▅▇▆▅▄▆▆▇▇█▇▇█▇▇▇█
auroc,▁▆▆▅▇▆▅▃▆▆▇▇█▆▆█▇▇▇█
avg_train_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▁▂▂▁▂▁▁
avg_val_loss,█▂▁▁▁▂▁▂▂▂▁▁▁▂▂▁▁▁▁▁
sensitivity,▁▅▃▅█▇▃▂▅▄▆▆▆▃▄▆▇▇▆█
specificity,▂▄▇▂▁▂▅▅▄▆▄▇▆██▇▅▄▆▅
accuracy,0.92
auprc,0.98054
auroc,0.9816
avg_train_loss,0.24223


Finished training


## SWEEP

In [ ]:

# Define the sweep configuration
sweep_config = {
    "method": "random",  # or "grid", or "bayes"
    "parameters": {
        "model_type": {
            "values": ["AutoMorphModel", "Task2EfficientNetB0", "Task1ConvNeXt"]
        },
        "lr": {
            "values": [1e-3]
        },
        "epochs": {
            "values": [20]
        },
        "batch_size": {
            "values": [16, 32]
        },
        "p_gaussblur": {
            "values": [0, 0.3]
        },
        "p_equalize": {
            "values": [0, 0.3]
        },
        "p_clahe": {
            "values": [0, 0.3]
        },
        "p_horizontalflip": {
            "values": [0, 0.3]
        },
        "rotation": {
            "values": [15, 30]
        },
        "p_affine": {
            "values": [0, 0.3]
        },
        "loss_weight": {
            "values": [0.5, 0.7]
        },
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="task2")

# Start the sweep
wandb.agent(sweep_id, function=train_model)